<a href="https://colab.research.google.com/github/OliverGriffiths287/OliverGriffiths287.github.io/blob/main/python_code/scraped_prempoints.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import re
import json
import pandas as pd

In [ ]:
from mmap import PAGESIZE
url = "https://www.tntsports.co.uk/football/premier-league/2022-2023/standings.shtml"

req = requests.get(url)
page_src = req.text

start_index = page_src.index('lg:col-start-1')
page_src = page_src[start_index:]

page_src = page_src.split('</table></div>')[0]


page_src

'lg:col-start-1" data-testid="column"><div class="flex items-center"><div class="flex items-center"><h1 class="lines-3 lg:lines-2 text-secondary-01 section-title-2 lg:section-title-1">Premier League Table 2022/2023</h1></div></div><div class="mt-5"><div data-testid="organism-score-center-filters"><div class="flex lg:flex-row justify-between"><div class="flex min-w-0 lg:space-x-9 -lg:flex-col -lg:space-y-5"><div class="relative min-w-0" data-testid="atom-dropdown-filter" data-analytics-id="season-dropdown"><button class="flex w-full items-center justify-between focus:outline-none h-9 border-b pb-[10.5px] pt-[6.5px] border-onLight-05 text-onLight-02 hover:border-onLight-04 active:border-onLight-04" data-testid="atom-dropdown-filter-button" id="headlessui-listbox-button-:R3dlimd6:" type="button" aria-haspopup="listbox" aria-expanded="false" data-headlessui-state=""><div class="flex items-center truncate"><span class="select-none truncate caption-1">2022/2023</span></div><svg xmlns="http:/

In [ ]:
from os import name
# Find all team entries using regex
team_entries = re.findall(r'md:block">(.*?)</a><a', page_src)

cleaned_teams = [name.replace('United', 'Utd')
                     .replace('Manchester', 'Man')
                     .replace(' &amp; Hove Albion', '')
                     .replace('Hotspur', '')
                     .replace('Wolverhampton Wanderers', 'Wolves')
                 for name in team_entries]

dfteams = pd.DataFrame(cleaned_teams, columns=['team'])

cleaned_teams
dfteams = pd.DataFrame(cleaned_teams)
dfteams.rename(columns={0: 'team'}, inplace=True)
dfteams

,team
0,Man City
1,Arsenal
2,Man Utd
3,Newcastle Utd
4,Liverpool
5,Brighton
6,Aston Villa
7,Tottenham
8,Brentford
9,Fulham


In [ ]:
points_total = re.findall(r'!font-bold lg:caption-1 lg:h-14 lg:px-1 lg:h-14 lg:caption-1 w-8 lg:w-11" data-testid="table-cell-value">(.*?)</td><td', page_src)
points_total = [int(points) for points in points_total]

points_total

cleaned_points = points_total[1::2]
cleaned_points_df = pd.DataFrame(cleaned_points, columns=['points'])
cleaned_points_df

points_23 = pd.concat([dfteams, cleaned_points_df], axis=1)
points_23

points_23.to_csv('prempoints_23.csv', index=False)



The following section looks at using the code above for one singular season but for all ten desired seasons through loops, by implementing a list of seasons into the url to get the data for each season in one dataframe.

In [ ]:
import requests
import re
import pandas as pd

base_url = "https://www.tntsports.co.uk/football/premier-league/{season}/standings.shtml"
seasons = range(2013, 2023)  # This will cover 10 seasons from 2013-2014 to 2022-2023

all_data = []

for season in seasons:
    season_str = f"{season}-{season+1}"
    url = base_url.format(season=season_str)

    req = requests.get(url)
    page_src = req.text

    start_index = page_src.index('lg:col-start-1')
    page_src = page_src[start_index:]
    page_src = page_src.split('</table></div>')[0]

    team_entries = re.findall(r'md:block">(.*?)</a><a', page_src)
    cleaned_teams = [name.replace('United', 'Utd')
                         .replace('Manchester', 'Man')
                         .replace(' &amp; Hove Albion', '')
                         .replace('Hotspur', '')
                         .replace('Wolverhampton Wanderers', 'Wolves')
                     for name in team_entries]

    points_total = re.findall(r'!font-bold lg:caption-1 lg:h-14 lg:px-1 lg:h-14 lg:caption-1 w-8 lg:w-11" data-testid="table-cell-value">(.*?)</td><td', page_src)
    points_total = [int(points) for points in points_total]
    cleaned_points = points_total[1::2]

    season_data = pd.DataFrame({'team': cleaned_teams, 'points': cleaned_points, 'season': season_str})
    all_data.append(season_data)

    print(f"Processed season {season_str}")

final_df = pd.concat(all_data, ignore_index=True)
final_df

Processed season 2013-2014
Processed season 2014-2015
Processed season 2015-2016
Processed season 2016-2017
Processed season 2017-2018
Processed season 2018-2019
Processed season 2019-2020
Processed season 2020-2021
Processed season 2021-2022
Processed season 2022-2023


,team,points,season
0,Man City,86,2013-2014
1,Liverpool,84,2013-2014
2,Chelsea,82,2013-2014
3,Arsenal,79,2013-2014
4,Everton,72,2013-2014
...,...,...,...
195,Nottingham Forest,38,2022-2023
196,Everton,36,2022-2023
197,Leicester City,34,2022-2023
198,Leeds Utd,31,2022-2023
